## Training a Chatbot on top of a T5 model

This trains on top of Flan-T5-base using the Alpaca dataset.  It needs to be run on a machine with a 24G GPU or bigger.  With bigger GPUs you can train over more data and with bigger batch sizes.  The default parameters work for a 24G GPU (like an A10) as they are the easiest to come by.  If the GPU is too small the Jupyter kernel will crash. To see GPU memory usage use the command nvidia-smi.

In [1]:
pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade

  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 3.2 MB/s eta 0:00:0000:0100:01m
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 777.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 2.4 MB/s eta 0:00:0000:0100:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 

  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=2e1677306204e56aa5ac20595f756e0f6fa619dfc6bb911670dd7bc55babafc0
  Stored in directory: /Users/costello/Library/Caches/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
  Attempting uninstall: transformers
    Found existi

This installed the basic packages. transformers is the core piece of large langauge models.  Rouge is something that scores outputs *Recall-Oriented Understudy for Gisting Evaluation* datasets is a package for handling our dataset, tensorboard shows progress as a bar.

In [2]:
!git clone https://github.com/tatsu-lab/stanford_alpaca.git
!pip install   protoc   --upgrade
!pip install   protobuf   --upgrade
!pip install protobuf==3.20.*
!pip install evaluate
!mkdir data

fatal: destination path 'stanford_alpaca' already exists and is not an empty directory.
ERROR: Could not find a version that satisfies the requirement protoc (from versions: none)
ERROR: No matching distribution found for protoc
  Using cached protobuf-4.22.1-cp37-abi3-macosx_10_9_universal2.whl (397 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.7 MB/s eta 0:00:00
mkdir: data: File exists


We clone the alpaca git repository.  evaluate is needed for the rouge metric.

In [3]:
import json
with open("stanford_alpaca/alpaca_data.json") as file:
    jj = json.loads(file.read() )

jjout = []
for i in jj:
    jjout.append({"input": i["instruction"] + " " + i["input"], "output": i["output"]})


with open("data/train.json", "w") as file:
    file.write(json.dumps(jjout[100:15000])) # Trim to 15000 for small GPUs.  

with open("data/test.json", "w") as file:
    file.write(json.dumps(jjout[0:100]))


from random import randrange
sample = jj[randrange(len(jj))]
print(sample)


{'instruction': 'Explain the significance of the provided historical event.', 'input': 'Magna Carta', 'output': 'The Magna Carta was a charter of rights agreed to by King John of England in 1215. The document limited the powers of the monarchy and served as important precedent for the rule of law. It established unprecedented rights of the individual, such as the right to a fair trial and freedom from arbitrary imprisonment. The Magna Carta is widely considered the inspiration for the US Constitution and other important documents throughout history.'}


We load the inputs and outputs from the Alpaca dataset into a JSON dict.  We combine the fields input and instruction together, and write out the dataset.  We make the first 100 be our test set.  We ouput an example so we know what the data looks like.

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-large
tokenizer = AutoTokenizer.from_pretrained(model_id)



from datasets import load_dataset


data_files = {"train": "train.json", "test": "test.json"}
dataset = load_dataset("data", data_files=data_files)


#dataset = load_dataset("json", data_files="jj1000.json")



# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.

tokenized_inputs = dataset['train'].map(lambda x: tokenizer(x["input"], truncation=True), batched=True, remove_columns=["input", "output"])
#tokenized_inputs = list(map(lambda x: tokenizer( (x[ "instruction"] + " " + x["input"])[0:510]  ), jj))

max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset['train'].map(lambda x: tokenizer(x["output"], truncation=True), batched=True, remove_columns=[  "input", "output" ])

#tokenized_targets = list(map(lambda x: tokenizer(x["output"][0:510]), jj))
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/json/data-0e6b99dc5ee550a6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Max source length: 321


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Max target length: 512


We load the model from huggingface.  We load the files we made in to a dataset object and measure the max lengths of the input and output, which we will need later. 

In [5]:


def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["input"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["output"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


We write a function that tokenizes, that is, turns each word into a vector of numbers.   This is where we use the max lengths.

In [6]:

from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
This is some code so we can see our progress as we go.  It measures the rouge score over the test set.

In [7]:

from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-alpaca"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)


This is where we set the parameters for our training.  The batch size controls how many we samples we process at a time. Larger values make things run faster at the cost of using more memory. The learning rate is usually about 1e-5. We run over the data 5 times (epochs).  We save our results every 500 steps, and only keep two copies.  The hub fields control what we push to HiggingFace, but we have not logged in so this will no work.

The give the trainer the test and train datasets, the function to computre our rouge score, and the model.

We hope to get our training loss to about 2.02, our validation loss to a similar number 1.73, and the Rouge scores to 40, 36, 37.  Higher is better for scores and worse for loss.

In [8]:

trainer.train()


/home/ubuntu/.local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.313400,1.932085,39.362700,23.093300,36.286700,37.048100,15.620000
2,2.192800,1.834956,38.928100,22.787300,35.742100,36.393700,15.650000
3,2.114200,1.780069,39.119000,23.221900,35.656100,36.577200,15.580000
4,2.054600,1.746920,38.917200,24.386900,36.085200,37.089900,15.460000
5,2.026800,1.733723,38.769300,24.420000,36.230500,36.924800,15.360000


TrainOutput(global_step=9375, training_loss=2.14437248046875, metrics={'train_runtime': 5067.752, 'train_samples_per_second': 14.799, 'train_steps_per_second': 1.85, 'total_flos': 3.21983071488e+16, 'train_loss': 2.14437248046875, 'epoch': 5.0})

In [26]:
def f(str):
    inputs = tokenizer(str, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_length=200,
                             num_beams=10,
                             repetition_penalty=2.5,
                             length_penalty=1.0,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             do_sample = True,
                             temperature = 1.5,
                             top_k = 50,
                             top_p = 0.95)
    
    return tokenizer.decode(outputs[0][1:-1])

In [27]:
f("What is the capital of Spain?")

'The capital of Spain is Madrid.'

This will launch a web page on port 8080.

In [30]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import time

hostName = "0.0.0.0"
serverPort = 8080

class MyServer(BaseHTTPRequestHandler):
    history = ""
    def do_GET(self):
        print("got query", self.path)
        print("Init history is ", MyServer.history)
        if MyServer.history == None:
            print("set history")
            MyServer.history = "" 
        if  self.path[0:6] == '/query':
            q = self.path[13:]
            q = q.replace("+", " ")
            q = q.replace("%3F", "?")
            print("q", q)
            ans = f(q)
            print("ans", ans)
            print("history is ", MyServer.history)
            MyServer.history = MyServer.history + "<p><i>" + q + "</i><p>" + ans
        else:
            ans = ""
            q = "" 

        page = """<html><head><title> A ChatBot </title>
<meta charset="utf-8">
	<title>CSS Zen Garden: The Beauty of CSS Design</title>

	<link rel="stylesheet" media="screen" href="http://www.csszengarden.com/214/214.css?v=8may2013">
	<link rel="alternate" type="application/rss+xml" title="RSS" href="http://www.csszengarden.com/zengarden.xml">

	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<meta name="author" content="Dave Shea">
	<meta name="description" content="A demonstration of what can be accomplished visually through CSS-based design.">
	<meta name="robots" content="all">
    <script> function fun()
{
 window.scrollTo(0, document.body.scrollHeight);
}
</script>
</head><body onload="fun">

<body id="css-zen-garden">
<div class="page-wrapper">

	<section class="intro" id="zen-intro">
		<header role="banner">
			<h1>A Chatbot</h1>
</header>

		<div class="summary" id="zen-summary" role="article">
			<p> """ + MyServer.history  + """ <p> 
 <form action="query" method="get" class="form-example">
  <div class="form-example">
      <input type="text" name="input" id="input" required  size="200">
  </div>
</div>
</form> </body></html> """
       

        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()

        

        self.wfile.write(bytes(page, "utf-8"))

if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://0.0.0.0:8080
got query /query?input=Generate+a+brainstorm+list+of+things+related+to+the+topic+of+artificial+intelligence.
Init history is  
q Generate a brainstorm list of things related to the topic of artificial intelligence.
ans 1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be Utilized to Improve Customer Service? 5. How Should AI Be Used for Education? 6. How Do AI Technologies Impact Business in the 21st Century? 7. How Will AI Help Businesses In The 2025 World? 8. How Big is AI's Impact on Banking, Retail, and Consumer Markets? 9. How Ease of Implementing AI has Shaped Our Future. 10. How We Can Make Achieving Our Vision Possible?
history is  
got query /
Init history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Bene

76.21.126.113 - - [17/Mar/2023 20:49:34] "GET /query?input=Generate+a+brainstorm+list+of+things+related+to+the+topic+of+artificial+intelligence. HTTP/1.1" 200 -
76.21.126.113 - - [17/Mar/2023 20:49:34] "GET / HTTP/1.1" 200 -


got query /favicon.ico
Init history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be Utilized to Improve Customer Service? 5. How Should AI Be Used for Education? 6. How Do AI Technologies Impact Business in the 21st Century? 7. How Will AI Help Businesses In The 2025 World? 8. How Big is AI's Impact on Banking, Retail, and Consumer Markets? 9. How Ease of Implementing AI has Shaped Our Future. 10. How We Can Make Achieving Our Vision Possible?


76.21.126.113 - - [17/Mar/2023 20:49:34] "GET /favicon.ico HTTP/1.1" 200 -


got query /query?input=What+is+the+capital+of+Spain%3F
Init history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be Utilized to Improve Customer Service? 5. How Should AI Be Used for Education? 6. How Do AI Technologies Impact Business in the 21st Century? 7. How Will AI Help Businesses In The 2025 World? 8. How Big is AI's Impact on Banking, Retail, and Consumer Markets? 9. How Ease of Implementing AI has Shaped Our Future. 10. How We Can Make Achieving Our Vision Possible?
q What is the capital of Spain?
ans The capital of Spain is Madrid.
history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be 

...

got query /query?input=What+are+the+main+differences+between+aerobic+and+anaerobic+exercise%3F+
Init history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be Utilized to Improve Customer Service? 5. How Should AI Be Used for Education? 6. How Do AI Technologies Impact Business in the 21st Century? 7. How Will AI Help Businesses In The 2025 World? 8. How Big is AI's Impact on Banking, Retail, and Consumer Markets? 9. How Ease of Implementing AI has Shaped Our Future. 10. How We Can Make Achieving Our Vision Possible?<p><i>What is the capital of Spain?</i><p>The capital of Spain is Madrid.<p><i>What is the capital of Spain?</i><p>The capital of Spain is Madrid.<p><i>What are the best VC firms?</i><p>The best VC firms are those that have the expertise and resources to launch innovative products 

76.21.126.113 - - [17/Mar/2023 20:54:02] "GET /query?input=What+are+the+main+differences+between+aerobic+and+anaerobic+exercise%3F+ HTTP/1.1" 200 -


got query /favicon.ico
Init history is  <p><i>Generate a brainstorm list of things related to the topic of artificial intelligence.</i><p>1. Artificial Intelligence (AI) - What Is It? 2. How Does AI Technology Work? 3. What Are the Benefits and Consequences of AI? 4. How Can AI be Utilized to Improve Customer Service? 5. How Should AI Be Used for Education? 6. How Do AI Technologies Impact Business in the 21st Century? 7. How Will AI Help Businesses In The 2025 World? 8. How Big is AI's Impact on Banking, Retail, and Consumer Markets? 9. How Ease of Implementing AI has Shaped Our Future. 10. How We Can Make Achieving Our Vision Possible?<p><i>What is the capital of Spain?</i><p>The capital of Spain is Madrid.<p><i>What is the capital of Spain?</i><p>The capital of Spain is Madrid.<p><i>What are the best VC firms?</i><p>The best VC firms are those that have the expertise and resources to launch innovative products or services. Some of these companies can include Microsoft, Apple, Google

76.21.126.113 - - [17/Mar/2023 20:54:02] "GET /favicon.ico HTTP/1.1" 200 -


Server stopped.


In [31]:
f("Calculate the number of hours in 3 days.")


'The total number of hours in 3 days is 24 hours.'